In [16]:
!pip3 install -U -r requirements.txt

     |████████████████████████████████| 10.7 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 16.1 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 7.3 MB 6.7 MB/s eta 0:00:01     |███████████████████             | 4.3 MB 520 kB/s eta 0:00:06     |████████████████████████▊       | 5.6 MB 520 kB/s eta 0:00:04
     |████████████████████████████████| 15.6 MB 3.0 MB/s eta 0:00:01    |█                               | 471 kB 2.3 MB/s eta 0:00:07     |██▍                             | 1.2 MB 2.3 MB/s eta 0:00:07     |███████████▏                    | 5.5 MB 1.7 MB/s eta 0:00:06     |███████████████████▌            | 9.5 MB 772 kB/s eta 0:00:08     |████████████████████████████████| 15.6 MB 3.0 MB/s eta 0:00:01


  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.4
    Uninstalling pandas-1.1.4:
      Successfully uninstalled pandas-1.1.4
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.3
    Uninstalling matplotlib-3.3.3:
      Successfully uninstalled matplotlib-3.3.3
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [17]:
### import libraries
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.models import Sequential

from keras.applications.densenet import DenseNet121
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import load_model
from keras.utils.vis_utils import plot_model

import pandas as pd
import numpy as np
from pathlib import Path

import matplotlib
matplotlib.use("Agg") # set the matplotlib backend so figures can be saved in the background
 
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt


In [4]:
dtrain=pd.read_csv("CheXpert-v1.0-small/train.csv", nrows=1000)
dtrain = dtrain.fillna(0)
# import valid data info from valid.csv
dnew=pd.read_csv("CheXpert-v1.0-small/valid.csv", nrows=100)
dnew = dnew.fillna(0)

#add dnew to dtrain to re-split since valid data in data set is very small
dtrain = dtrain.append(dnew)

#pre-process data: remove Lateral images
dtrain = dtrain[~dtrain[dtrain.columns[3]].str.contains("Lateral")]
#pre-process data: drop selected features - only images as inputs
dtrain = dtrain.drop(["Sex", "Age", "Frontal/Lateral", "AP/PA"], axis=1)
print(dtrain.shape)
dtrain.describe().transpose()

(110, 15)


,count,mean,std,min,25%,50%,75%,max
No Finding,110.0,0.181818,0.387460,0.0,0.0,0.0,0.0,1.0
Enlarged Cardiomediastinum,110.0,0.109091,0.456260,-1.0,0.0,0.0,0.0,1.0
Cardiomegaly,110.0,0.109091,0.391315,-1.0,0.0,0.0,0.0,1.0
Lung Opacity,110.0,0.463636,0.553177,-1.0,0.0,0.0,1.0,1.0
Lung Lesion,110.0,0.045455,0.209252,0.0,0.0,0.0,0.0,1.0
Edema,110.0,0.118182,0.351454,-1.0,0.0,0.0,0.0,1.0
Consolidation,110.0,-0.009091,0.370852,-1.0,0.0,0.0,0.0,1.0
Pneumonia,110.0,0.000000,0.191565,-1.0,0.0,0.0,0.0,1.0
Atelectasis,110.0,0.154545,0.544975,-1.0,0.0,0.0,0.0,1.0
Pneumothorax,110.0,0.109091,0.367123,-1.0,0.0,0.0,0.0,1.0


In [5]:
# dealing with uncertanty (-1) values
dtrain = dtrain.replace(-1,1)
dtrain.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
No Finding,110.0,0.181818,0.387460,0.0,0.0,0.0,0.0,1.0
Enlarged Cardiomediastinum,110.0,0.218182,0.414902,0.0,0.0,0.0,0.0,1.0
Cardiomegaly,110.0,0.163636,0.371638,0.0,0.0,0.0,0.0,1.0
Lung Opacity,110.0,0.518182,0.501956,0.0,0.0,1.0,1.0,1.0
Lung Lesion,110.0,0.045455,0.209252,0.0,0.0,0.0,0.0,1.0
Edema,110.0,0.136364,0.344745,0.0,0.0,0.0,0.0,1.0
Consolidation,110.0,0.136364,0.344745,0.0,0.0,0.0,0.0,1.0
Pneumonia,110.0,0.036364,0.188050,0.0,0.0,0.0,0.0,1.0
Atelectasis,110.0,0.318182,0.467902,0.0,0.0,0.0,1.0,1.0
Pneumothorax,110.0,0.145455,0.354172,0.0,0.0,0.0,0.0,1.0


In [6]:
### split data into train/valid/test
dtrain = dtrain.sample(frac=1)
dvalid_size = round(0.1*dtrain.shape[0])
dtest_size = dvalid_size
dtr = dtrain[0:dtrain.shape[0]-dvalid_size-dtest_size+1]
dv = dtrain[dtrain.shape[0]-dvalid_size-dtest_size:dtrain.shape[0]-dvalid_size+1]
dte = dtrain[dtrain.shape[0]-dvalid_size:dtrain.shape[0]+1]

print(dv.shape)

(12, 15)


In [7]:
### data generation for Keras 
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255.)
valid_datagen=ImageDataGenerator(rescale=1./255.)

target_size = (224,224)
#target_size = (299,299)
#target_size = (75,75)
train_generator=train_datagen.flow_from_dataframe(dataframe=dtr, directory=None , x_col="Path", y_col=list(dtr.columns[1:15]), class_mode="other", target_size=target_size, batch_size=32)
valid_generator=valid_datagen.flow_from_dataframe(dataframe=dv, directory=None , x_col="Path", y_col=list(dv.columns[1:15]), class_mode="other", target_size=target_size, batch_size=32)
test_generator=test_datagen.flow_from_dataframe(dataframe=dte, directory=None , x_col="Path", y_col=list(dte.columns[1:15]), class_mode="other", target_size=target_size, shuffle = False, batch_size=1)

Found 89 validated image filenames.
Found 12 validated image filenames.
Found 11 validated image filenames.


In [12]:
### model architecture design/selection
# create the base pre-trained model
base_model = DenseNet121(include_top = False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(14, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional DenseNet layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)

#model training
adam = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer= adam, loss='binary_crossentropy', metrics=['accuracy'])

#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
#print(model.summary())

In [13]:

### fit model 
num_epochs = 3
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

model_H = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=num_epochs)
# save model
model.save("model_DenseNet121_TF.h5")

Epoch 1/3
2/2 [==============================] - 64s 5s/step - loss: 0.7853 - accuracy: 0.0780
Epoch 2/3
2/2 [==============================] - 12s 6s/step - loss: 0.6102 - accuracy: 0.1667
Epoch 3/3
2/2 [==============================] - 11s 6s/step - loss: 0.5139 - accuracy: 0.3789


In [18]:
# load my trained model 
model_F = load_model('model_DenseNet121_TF.h5')
num_epochs = 3
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [19]:
### prediction and performance assessment
test_generator.reset()
pred=model.predict_generator(test_generator, steps=STEP_SIZE_TEST)
pred_bool = (pred >= 0.5)

y_pred = np.array(pred_bool,dtype =int)

dtest = dte.to_numpy()
y_true = np.array(dtest[:,1:15],dtype=int)

print(classification_report(y_true, y_pred,target_names=list(dtr.columns[1:15])))

/usr/local/lib/python3.9/site-packages/keras/engine/training.py:1976: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


                            precision    recall  f1-score   support

                No Finding       0.00      0.00      0.00         2
Enlarged Cardiomediastinum       0.00      0.00      0.00         2
              Cardiomegaly       0.00      0.00      0.00         3
              Lung Opacity       0.29      0.67      0.40         3
               Lung Lesion       0.00      0.00      0.00         0
                     Edema       0.00      0.00      0.00         2
             Consolidation       0.00      0.00      0.00         1
                 Pneumonia       0.00      0.00      0.00         1
               Atelectasis       0.00      0.00      0.00         3
              Pneumothorax       0.00      0.00      0.00         2
          Pleural Effusion       0.00      0.00      0.00         5
             Pleural Other       0.00      0.00      0.00         0
                  Fracture       0.00      0.00      0.00         0
           Support Devices       0.00      0.00

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  
